## Generate a large amount of training data for picking net

In [1]:
# import liblrary
import numpy as np
import h5py
import random
import os
import sys
sys.path.insert(1, '../')
from demos.utils import *

# Make a new dir to save the picking error
directory_path = "../train_data_picking/"

# Create the directory, including any necessary parent directories
os.makedirs(directory_path, exist_ok=True)

2024-11-16 16:04:09.294877: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 16:04:09.294907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 16:04:09.295782: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 16:04:09.300324: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-16 16:04:09.813340: W tensorflow/compiler/tf2

## Data path and load data

In [4]:
# path
signal_id_path = '../data/signalid_random_10w.npy'
TXED_path = os.getenv('HOME')+'/Yang/Data/Earthquake_data/TXED/TXED_0913.h5' # Here, you may need to change the path of TXED
out_path = '../train_data_picking'

# load
Dection_signal_id = np.load(signal_id_path, 'r')
f_txed = h5py.File(TXED_path, 'r')

## Generate arrivals, obtain waveforms, and build labels

In [5]:
# obtain the P- and S-wave arrivals
P_arrival_list = []
S_arrival_list = []
print('-----------arrival time calculation begin-------------------')
for key in Dection_signal_id:
    if key in f_txed:   
        dataset = f_txed.get(key)
        P_arrival_list.append(int(dataset.attrs['p_arrival_sample']))
        S_arrival_list.append(int(dataset.attrs['s_arrival_sample']))
P_arrival_list = np.array(P_arrival_list)
S_arrival_list = np.array(S_arrival_list)
P_phase_label_indx = P_arrival_list
S_phase_label_indx = S_arrival_list
P_phase_label_indx = np.reshape(P_phase_label_indx, [P_phase_label_indx.shape[0], 1])
S_phase_label_indx = np.reshape(S_phase_label_indx, [S_phase_label_indx.shape[0], 1])
phase_label_indx = np.concatenate([P_phase_label_indx, S_phase_label_indx], axis=-1)
print('-----------arrival time calculation end-------------------')
print(f'shape of arrivals: {P_arrival_list.shape}')


# apply the bandpass and normalize to each waveform
signal_list = []
print('-----------signal format convert begin-------------------')
for key in Dection_signal_id:
    if key in f_txed:   
        dataset = f_txed.get(key)
        datas = dataset['data']
        datas = np.array(datas)
        datas_0 = butter_bandpass_filter_zi(datas[:,0], 1, 45, 100, order=3)
        datas_1 = butter_bandpass_filter_zi(datas[:,1], 1, 45, 100, order=3)
        datas_2 = butter_bandpass_filter_zi(datas[:,2], 1, 45, 100, order=3)
        datas = np.vstack([datas_0, datas_1, datas_2])
        signal_list.append(datas) 
signal_values = np.array(signal_list)
bp_signal= np.transpose(signal_values, [0, 2, 1])


#Normalized trace-by-trace
max_values_per_event = np.max(bp_signal, axis=1)
# Normalize each component of each event by dividing by its maximum value
normalized_phase_data = bp_signal / max_values_per_event[:, np.newaxis, :]
print('-----------signal format convert finish-------------------')
print(bp_signal.shape)

p_wave_label = []
s_wave_label = []
for i in range (normalized_phase_data.shape[0]):
    # Example usage:
    p_indx = P_arrival_list[i]  # Arrival time of P-wave (in seconds)
    s_indx = S_arrival_list[i]  # Arrival time of S-wave (in seconds)    
    sample_num = 6000  # Sampling rate of seismic signal (in Hz)
    #print(f'Processed_num:{i}\t P_arrival_time: {p_indx}\t S_arrival_time: {s_indx}\t sample_num: {sample_num}')
    if i % 5000 == 0:
        print(f'Processed_num: {i}\t P_arrival_time: {p_indx}\t S_arrival_time: {s_indx}\t sample_num: {sample_num}')
    
    # Generate labels for P-wave and S-wave first arrival picking
    p_labels, s_labels = generate_first_arrival_labels(p_indx, s_indx, sample_num)
    p_wave_label.append(p_labels)  
    s_wave_label.append(s_labels)
p_wave_label = np.array(p_wave_label)
s_wave_label = np.array(s_wave_label)
phase_label = np.concatenate([np.reshape(p_wave_label, [p_wave_label.shape[0], p_wave_label.shape[1], 1]), \
                              np.reshape(s_wave_label, [s_wave_label.shape[0], s_wave_label.shape[1], 1])], axis=-1)

-----------arrival time calculation begin-------------------
-----------arrival time calculation end-------------------
shape of arrivals: (100000,)
-----------signal format convert begin-------------------
-----------signal format convert finish-------------------
(100000, 6000, 3)
Processed_num: 0	 P_arrival_time: 99	 S_arrival_time: 616	 sample_num: 6000
Processed_num: 5000	 P_arrival_time: 598	 S_arrival_time: 741	 sample_num: 6000
Processed_num: 10000	 P_arrival_time: 98	 S_arrival_time: 328	 sample_num: 6000
Processed_num: 15000	 P_arrival_time: 99	 S_arrival_time: 1267	 sample_num: 6000
Processed_num: 20000	 P_arrival_time: 298	 S_arrival_time: 1749	 sample_num: 6000
Processed_num: 25000	 P_arrival_time: 299	 S_arrival_time: 940	 sample_num: 6000
Processed_num: 30000	 P_arrival_time: 898	 S_arrival_time: 1214	 sample_num: 6000
Processed_num: 35000	 P_arrival_time: 699	 S_arrival_time: 1360	 sample_num: 6000
Processed_num: 40000	 P_arrival_time: 598	 S_arrival_time: 1061	 sample_

## Save results

In [6]:
np.save(f'{out_path}EQ_detection_phase_label.npy', phase_label)
np.save(f'{out_path}EQ_detection_waveforms.npy', normalized_phase_data)